In [376]:
import pandas as pd
import numpy as np

In [377]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [378]:
## making Target column
train['Target']=train.LastWorkingDate.isnull().map(lambda y : 0 if y == True else 1)
train

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2,0
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2,0
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,1
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1,0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2017-08-01,2788,30,Male,C27,Master,70254,2017-06-08,NaN,2,2,740280,3,0
19100,2017-09-01,2788,30,Male,C27,Master,70254,2017-06-08,NaN,2,2,448370,3,0
19101,2017-10-01,2788,30,Male,C27,Master,70254,2017-06-08,NaN,2,2,0,2,0
19102,2017-11-01,2788,30,Male,C27,Master,70254,2017-06-08,NaN,2,2,200420,2,0


In [379]:
# Last Reporting month - Employee data
train["MMM-YY"]=pd.to_datetime(train["MMM-YY"])

# joining data
train["Dateofjoining"]=pd.to_datetime(train.Dateofjoining)
train["J_Month"] = train["Dateofjoining"].dt.month
train["J_Year"] = train["Dateofjoining"].dt.year

# Experience column 
train["Exp"] = ((train["MMM-YY"]-train["Dateofjoining"])//np.timedelta64(1, 'M')).map(lambda y : 0 if y < 0 else y )

# check if any designation change 
train["Change_in_Desigination"] =  train["Designation"] - train["Joining Designation"]

# Any Chnage in salary
train['Salary increased'] = (train.groupby('Emp_ID')['Salary'].diff().gt(0)).map(lambda y: 1 if y == True else 0)

train['Total Business Value'].fillna(0)

# cat colums 
train['Education_Level'] = train['Education_Level'].map({'College': 1,'Bachelor': 2,'Master': 3})
train['Gender'] = train['Gender'].map({'Male': 0,'Female': 1})
train

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,Target,J_Month,J_Year,Exp,Change_in_Desigination,Salary increased
0,2016-01-01,1,28,0,C23,3,57387,2015-12-24,NaN,1,1,2381060,2,0,12,2015,0,0,0
1,2016-02-01,1,28,0,C23,3,57387,2015-12-24,NaN,1,1,-665480,2,0,12,2015,1,0,0
2,2016-03-01,1,28,0,C23,3,57387,2015-12-24,2016-03-11,1,1,0,2,1,12,2015,2,0,0
3,2017-11-01,2,31,0,C7,3,67016,2017-11-06,NaN,2,2,0,1,0,11,2017,0,0,0
4,2017-12-01,2,31,0,C7,3,67016,2017-11-06,NaN,2,2,0,1,0,11,2017,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19099,2017-08-01,2788,30,0,C27,3,70254,2017-06-08,NaN,2,2,740280,3,0,6,2017,1,0,0
19100,2017-09-01,2788,30,0,C27,3,70254,2017-06-08,NaN,2,2,448370,3,0,6,2017,2,0,0
19101,2017-10-01,2788,30,0,C27,3,70254,2017-06-08,NaN,2,2,0,2,0,6,2017,3,0,0
19102,2017-11-01,2788,30,0,C27,3,70254,2017-06-08,NaN,2,2,200420,2,0,6,2017,4,0,0


In [380]:
group = train.groupby(['Emp_ID'])
group = group.last().sort_index().reset_index()
group.drop(['Total Business Value'], axis= 1,inplace= True)

In [381]:
group1=train.groupby('Emp_ID').agg({'Total Business Value':'sum'}).reset_index()
group1.drop(['Emp_ID'],axis= 1,inplace= True)

In [382]:
new_data =pd.concat([group,group1],axis=1,ignore_index=False) # Joining Both the Groups 

In [383]:
new_data['Target'] = new_data.pop('Target')
new_data

,Emp_ID,MMM-YY,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Quarterly Rating,J_Month,J_Year,Exp,Change_in_Desigination,Salary increased,Total Business Value,Target
0,1,2016-03-01,28,0,C23,3,57387,2015-12-24,2016-03-11,1,1,2,12,2015,2,0,0,1715580,1
1,2,2017-12-01,31,0,C7,3,67016,2017-11-06,None,2,2,1,11,2017,0,0,0,0,0
2,4,2017-04-01,43,0,C13,3,65603,2016-12-07,2017-04-27,2,2,1,12,2016,3,0,0,350000,1
3,5,2016-03-01,29,0,C9,1,46368,2016-01-09,2016-03-07,1,1,1,1,2016,1,0,0,120360,1
4,6,2017-12-01,31,1,C11,2,78728,2017-07-31,None,3,3,2,7,2017,4,0,0,1265000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376,2784,2017-12-01,34,0,C24,1,82815,2012-10-15,None,2,3,4,10,2012,61,1,0,21748820,0
2377,2785,2017-10-01,34,1,C9,1,12105,2017-08-28,2017-10-28,1,1,1,8,2017,1,0,0,0,1
2378,2786,2016-09-01,45,0,C19,1,35370,2015-07-31,2016-09-22,2,2,1,7,2015,13,0,0,2815090,1
2379,2787,2016-06-01,28,1,C20,3,69498,2015-07-21,2016-06-20,1,1,1,7,2015,10,0,0,977830,1


In [384]:
from sklearn import preprocessing ##Coverting categorical features into numerical using Label_Encoder

le=preprocessing.LabelEncoder()
new_data['City']=le.fit_transform(new_data['City'])

In [385]:
new_data

,Emp_ID,MMM-YY,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Quarterly Rating,J_Month,J_Year,Exp,Change_in_Desigination,Salary increased,Total Business Value,Target
0,1,2016-03-01,28,0,15,3,57387,2015-12-24,2016-03-11,1,1,2,12,2015,2,0,0,1715580,1
1,2,2017-12-01,31,0,26,3,67016,2017-11-06,None,2,2,1,11,2017,0,0,0,0,0
2,4,2017-04-01,43,0,4,3,65603,2016-12-07,2017-04-27,2,2,1,12,2016,3,0,0,350000,1
3,5,2016-03-01,29,0,28,1,46368,2016-01-09,2016-03-07,1,1,1,1,2016,1,0,0,120360,1
4,6,2017-12-01,31,1,2,2,78728,2017-07-31,None,3,3,2,7,2017,4,0,0,1265000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376,2784,2017-12-01,34,0,16,1,82815,2012-10-15,None,2,3,4,10,2012,61,1,0,21748820,0
2377,2785,2017-10-01,34,1,28,1,12105,2017-08-28,2017-10-28,1,1,1,8,2017,1,0,0,0,1
2378,2786,2016-09-01,45,0,10,1,35370,2015-07-31,2016-09-22,2,2,1,7,2015,13,0,0,2815090,1
2379,2787,2016-06-01,28,1,12,3,69498,2015-07-21,2016-06-20,1,1,1,7,2015,10,0,0,977830,1


In [386]:
new_data.columns

Index(['Emp_ID', 'MMM-YY', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'LastWorkingDate', 'Joining Designation',
       'Designation', 'Quarterly Rating', 'J_Month', 'J_Year', 'Exp',
       'Change_in_Desigination', 'Salary increased', 'Total Business Value',
       'Target'],
      dtype='object')

In [387]:
new_data.drop(['MMM-YY','LastWorkingDate','Dateofjoining'] ,axis= 1,inplace= True)

In [388]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
x=new_data.drop(['Target'],axis=1)
vif=pd.DataFrame()
vif['Column Names']=x.columns
vif['VIF']=[variance_inflation_factor(x.values,i) for i in range(len(x.columns))]
vif

C:\Users\Yash\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,Column Names,VIF
0,Emp_ID,4.036126
1,Age,37.526912
2,Gender,1.713685
3,City,3.936457
4,Education_Level,7.471081
5,Salary,13.014534
6,Joining Designation,inf
7,Designation,inf
8,Quarterly Rating,6.162445
9,J_Month,6.527711


In [389]:
new_data.drop(['Age','J_Year'],axis=1,inplace=True) # Drop 2 hightest variance inflation factor columns

In [390]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381 entries, 0 to 2380
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Emp_ID                  2381 non-null   int64
 1   Gender                  2381 non-null   int64
 2   City                    2381 non-null   int32
 3   Education_Level         2381 non-null   int64
 4   Salary                  2381 non-null   int64
 5   Joining Designation     2381 non-null   int64
 6   Designation             2381 non-null   int64
 7   Quarterly Rating        2381 non-null   int64
 8   J_Month                 2381 non-null   int64
 9   Exp                     2381 non-null   int64
 10  Change_in_Desigination  2381 non-null   int64
 11  Salary increased        2381 non-null   int64
 12  Total Business Value    2381 non-null   int64
 13  Target                  2381 non-null   int64
dtypes: int32(1), int64(13)
memory usage: 251.2 KB


In [391]:
from sklearn.utils import resample,shuffle
zeros=new_data[new_data.Target==0]
ones=new_data[new_data.Target==1]
up_zeros=resample(zeros,replace=True,n_samples=len(ones),random_state=127)

In [392]:
new_data1=pd.concat([up_zeros,ones])
new_data1=shuffle(new_data1)       #Shuffling the dataframe
new_data1=new_data1.reset_index(drop=True)
new_data1

,Emp_ID,Gender,City,Education_Level,Salary,Joining Designation,Designation,Quarterly Rating,J_Month,Exp,Change_in_Desigination,Salary increased,Total Business Value,Target
0,2324,0,10,3,41798,1,1,1,10,2,0,0,0,1
1,1573,1,27,2,62349,1,2,4,7,64,1,0,19301010,0
2,1954,0,28,1,49213,1,1,1,10,6,0,0,1382460,1
3,2038,1,26,2,17687,1,1,1,1,0,0,0,0,1
4,1212,0,28,1,47419,1,2,1,5,16,1,0,7414770,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3227,1316,1,20,2,75773,2,3,3,10,25,1,0,37833950,0
3228,1640,0,20,2,88169,3,3,2,11,11,0,0,11769770,1
3229,2693,0,26,1,58571,1,1,1,5,19,0,0,2204570,1
3230,1215,0,20,1,18088,1,1,1,5,2,0,0,0,1


In [393]:
X = new_data1.iloc[:,1:-1]       # Independent Variable
y = new_data1.iloc[:,-1]        # Dependent Variable [target variable]

In [394]:
print(X.shape)
print(y.shape)
print(test.shape)

(3232, 12)
(3232,)
(741, 1)


In [395]:
from sklearn.model_selection import train_test_split

#Split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=10)

In [396]:
print(X_train.shape,X_test.shape)

(2424, 12) (808, 12)


In [397]:
from sklearn.preprocessing import StandardScaler ## Using StandardScaler to scale the train and test sets into scaled versions.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [398]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score,classification_report,f1_score

In [399]:
model_LogisticRegression = LogisticRegression(C = 0.1, random_state = 42, solver = 'liblinear')
model_DecisionTree=DecisionTreeClassifier()
model_RandomForest=RandomForestClassifier()
model_GradientBoosting=GradientBoostingClassifier()
model_XGradientBoosting=XGBClassifier()

In [400]:
models = [model_LogisticRegression,model_DecisionTree,model_RandomForest,model_GradientBoosting,model_XGradientBoosting]
model_names = ['model_LogisticRegression','model_DecisionTree','model_RandomForest','model_GradientBoosting','model_XGradientBoosting']
i=0
for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    Y_pred=model.predict(X_test)
    print("Accuracy of The Model :",accuracy_score(y_test,Y_pred)*100)
    print("f1_score of The Model :",f1_score(y_test,Y_pred)*100)
    print(f"\n\n")

model_LogisticRegression
Accuracy of The Model : 75.4950495049505
f1_score of The Model : 77.13625866050808



model_DecisionTree
Accuracy of The Model : 91.58415841584159
f1_score of The Model : 91.56327543424318



model_RandomForest
Accuracy of The Model : 92.94554455445545
f1_score of The Model : 92.95426452410383



model_GradientBoosting
Accuracy of The Model : 86.13861386138613
f1_score of The Model : 86.72985781990522



model_XGradientBoosting
[15:55:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Yash\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accuracy of The Model : 93.06930693069307
f1_score of The Model : 93.01745635910224





In [401]:
test=pd.merge(test,new_data,on='Emp_ID')
test

,Emp_ID,Gender,City,Education_Level,Salary,Joining Designation,Designation,Quarterly Rating,J_Month,Exp,Change_in_Desigination,Salary increased,Total Business Value,Target
0,394,1,12,3,97722,2,4,3,6,77,2,0,27458220,0
1,173,0,20,1,56174,1,3,3,6,77,2,0,19866290,0
2,1090,0,4,1,96750,2,4,2,6,77,2,0,50382490,0
3,840,1,27,1,88813,1,4,2,7,76,3,0,17134040,0
4,308,0,24,3,188418,2,5,2,8,75,3,0,58024490,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,2134,0,21,1,116006,2,5,1,10,85,3,0,59696450,0
737,2255,0,17,1,133489,3,4,3,11,84,1,0,36398090,0
738,448,0,1,2,65389,2,4,3,4,79,2,0,30318780,0
739,1644,1,28,2,105513,2,4,1,4,79,2,0,28842760,0


In [402]:
test.drop(['Target','Emp_ID'],axis=1,inplace=True)
T_pred = model_XGradientBoosting.predict(test)
T_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [403]:
submission=pd.read_csv('sample_submission.csv')
submission['Target']=T_pred
submission.to_csv("Smaple.csv")

In [404]:
submission.Target.value_counts()

0    707
1     34
Name: Target, dtype: int64

In [405]:
# from sklearn.model_selection import GridSearchCV
# param_grid = { 
#     'n_estimators': [500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : list(range(16,26,2)),
#     'criterion' :['gini', 'entropy']
# }
# CV = GridSearchCV(estimator=model_RandomForest, param_grid=param_grid, cv= 5)
# CV.fit(X_train,y_train)
# Y_pred=CV.predict(X_test)
# print("Accuracy of The Model :",accuracy_score(y_test,Y_pred)*100)
# print("f1_score of The Model :",f1_score(y_test,Y_pred)*100)
# print("Classification Report",classification_report(y_test,Y_pred))

In [406]:
# best_param = CV.best_params_
# best_param